# 디바이스 자동 감지 테스트

이 노트북은 MPS/CUDA 자동 감지 시스템을 테스트합니다.

In [ ]:
import sys
from pathlib import Path

# 상위 디렉토리의 code를 import 경로에 추가
sys.path.append(str(Path.cwd().parent / 'code'))

from utils.device_utils import (
    get_system_info, 
    detect_cuda_devices, 
    detect_mps_device,
    get_optimal_device, 
    setup_device_config,
    print_device_summary
)
import torch

## 1. 시스템 정보 확인

In [ ]:
# 시스템 정보 수집
sys_info = get_system_info()

print("=== 시스템 정보 ===")
for key, value in sys_info.items():
    print(f"{key}: {value}")

## 2. 디바이스 감지

In [ ]:
# CUDA 디바이스 감지
print("=== CUDA 디바이스 ===")
cuda_devices = detect_cuda_devices()
if cuda_devices:
    for idx, device in cuda_devices.items():
        print(f"GPU {idx}: {device}")
        print(f"  - 메모리: {device.memory_gb:.2f} GB")
        print(f"  - Compute Capability: {device.compute_capability}")
else:
    print("CUDA 디바이스를 찾을 수 없습니다.")

# MPS 디바이스 감지
print("\n=== MPS 디바이스 ===")
mps_device = detect_mps_device()
if mps_device:
    print(f"MPS: {mps_device}")
else:
    print("MPS 디바이스를 찾을 수 없습니다.")

## 3. 최적 디바이스 선택

In [ ]:
# 최적 디바이스 자동 선택
device, device_info = get_optimal_device()

print(f"선택된 디바이스: {device}")
print(f"디바이스 정보: {device_info}")

# PyTorch 텐서 테스트
test_tensor = torch.randn(10, 10).to(device)
print(f"\n테스트 텐서 디바이스: {test_tensor.device}")
print(f"텐서 크기: {test_tensor.shape}")

## 4. 디바이스별 최적화 설정

In [ ]:
# 모델 크기별 최적화 설정 확인
model_sizes = ['small', 'base', 'large']

print(f"디바이스: {device_info}\n")

for size in model_sizes:
    config = setup_device_config(device_info, size)
    print(f"=== {size.upper()} 모델 설정 ===")
    print(f"배치 크기: {config.batch_size}")
    print(f"Mixed Precision: {config.mixed_precision}")
    print(f"FP16: {config.fp16}")
    print(f"Gradient Accumulation: {config.gradient_accumulation_steps}")
    print(f"DataLoader Workers: {config.num_workers}")
    print(f"Pin Memory: {config.pin_memory}")
    print(f"Gradient Checkpointing: {config.gradient_checkpointing}")
    print()

## 5. 환경변수 확인 (MPS)

In [ ]:
import os

# MPS 관련 환경변수 확인
if device_info.device_type == 'mps':
    print("=== MPS 환경변수 ===")
    mps_env_vars = [
        'PYTORCH_MPS_HIGH_WATERMARK_RATIO',
        'PYTORCH_ENABLE_MPS_FALLBACK'
    ]
    
    for var in mps_env_vars:
        value = os.environ.get(var, 'Not set')
        print(f"{var}: {value}")
else:
    print("MPS 디바이스가 아니므로 MPS 환경변수가 설정되지 않았습니다.")

## 6. 전체 디바이스 요약

In [ ]:
# 디바이스 요약 정보 출력
print_device_summary()

## 7. Trainer 통합 테스트

In [ ]:
# Trainer 클래스에서 디바이스 설정 테스트
from trainer import DialogueSummarizationTrainer
from utils.config_manager import ConfigManager

# 테스트용 설정
test_config = {
    'general': {
        'device': 'auto',  # 자동 감지
        'output_dir': 'test_output'
    },
    'model': {
        'name': 'gogamza/kobart-summarization',
        'size': 'base'
    },
    'training': {
        # 빈 설정 - 자동으로 채워질 예정
    }
}

# Trainer 초기화 (디바이스 설정만 테스트)
trainer = DialogueSummarizationTrainer(
    config=test_config,
    experiment_name='device_test'
)

print(f"\nTrainer 디바이스: {trainer.device}")
print(f"\n자동 적용된 학습 설정:")
for key, value in trainer.config['training'].items():
    if key in ['per_device_train_batch_size', 'mixed_precision', 'dataloader_num_workers', 'fp16']:
        print(f"  {key}: {value}")

## 8. 성능 벤치마크 (선택사항)

In [ ]:
import time

# 간단한 성능 테스트
def benchmark_device(device, size=1000):
    """디바이스 성능 벤치마크"""
    # 행렬 곱셈 테스트
    a = torch.randn(size, size).to(device)
    b = torch.randn(size, size).to(device)
    
    # Warm up
    for _ in range(10):
        _ = torch.matmul(a, b)
    
    # 실제 측정
    torch.cuda.synchronize() if device.type == 'cuda' else None
    start = time.time()
    
    for _ in range(100):
        c = torch.matmul(a, b)
    
    torch.cuda.synchronize() if device.type == 'cuda' else None
    end = time.time()
    
    return (end - start) / 100

# 벤치마크 실행
if device.type != 'cpu':  # GPU/MPS에서만 실행
    print("=== 성능 벤치마크 ===")
    for size in [500, 1000, 2000]:
        try:
            time_taken = benchmark_device(device, size)
            gflops = (2 * size**3) / (time_taken * 1e9)
            print(f"행렬 크기 {size}x{size}: {time_taken*1000:.2f}ms, {gflops:.2f} GFLOPS")
        except Exception as e:
            print(f"행렬 크기 {size}x{size}: 메모리 부족 또는 오류 - {e}")
else:
    print("CPU에서는 벤치마크를 건너뜁니다.")

## 결론

MPS/CUDA 자동 감지 시스템이 성공적으로 구현되었습니다:

1. **자동 디바이스 감지**: CUDA > MPS > CPU 우선순위로 최적 디바이스 선택
2. **플랫폼별 최적화**: 각 디바이스에 맞는 배치 크기, mixed precision, worker 수 등 자동 설정
3. **메모리 기반 조정**: GPU 메모리 크기에 따른 동적 설정 조정
4. **환경변수 설정**: MPS 사용 시 필요한 환경변수 자동 설정
5. **Trainer 통합**: DialogueSummarizationTrainer에 자동 통합되어 별도 설정 없이 사용 가능